In [1]:
! uv pip install langchain google-generativeai rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.3 environment at: C:\mlops\.venv
Resolved 86 packages in 4.26s
Uninstalled 1 package in 890ms
Installed 1 package in 1.40s
 - google-ai-generativelanguage==0.9.0
 + google-ai-generativelanguage==0.6.15


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

# Data Ingestion

In [3]:
from langchain_community.document_loaders import TextLoader

c:\mlops\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = TextLoader(r'C:\mlops\data\data.txt', encoding='utf8')
documents = loader.load()

In [5]:
documents[0].page_content[:500]

'Lufthansa flies back to profit\n\nGerman airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.\n\nIn a preliminary report, the airline announced net profits of 400m euros ($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering busin'

# Now divide it into chunks

In [6]:
! uv pip install langchain-text-splitters

Using Python 3.12.3 environment at: C:\mlops\.venv
Audited 1 package in 307ms


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

In [9]:
text_chunks = text_splitter.split_documents(documents)

In [10]:
text_chunks

[Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='Lufthansa flies back to profit\n\nGerman airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='In a preliminary report, the airline announced net profits of 400m euros ($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='at its US catering business. Last year, Lufthansa showed signs of recovery even as some European and US airlines were teetering on the brink of bankruptcy. The board of Lufthansa 

In [11]:
! uv pip install faiss-cpu

Using Python 3.12.3 environment at: C:\mlops\.venv
Audited 1 package in 27ms


In [12]:
! uv pip install sentence-transformers

Using Python 3.12.3 environment at: C:\mlops\.venv
Audited 1 package in 792ms


In [13]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS

In [14]:
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

class HFEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text: str):
        return self.model.encode(text).tolist()

    def embed_documents(self, texts: list[str]):
        return self.model.encode(texts).tolist()


In [15]:
embedder = HFEmbeddings()
faiss_store = FAISS.from_texts(
    [c.page_content for c in text_chunks],
    embedder
)


In [31]:
retreiver = faiss_store.as_retriever()

# Implement Data Retreival(Simple semantic search)

In [16]:
query = "What was Lufthansa's net profit in 2004 in Euros?"
docs = faiss_store.similarity_search(query, k=4)

for i, doc in enumerate(docs):
    print(f"Document {i+1}:\n{doc.page_content}\n")

Document 1:
Lufthansa flies back to profit

German airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.

Document 2:
In a preliminary report, the airline announced net profits of 400m euros ($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than

Document 3:
ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering

Document 4:
at its US catering business. Last year, Lufthansa showed signs of recovery even as some European and US airlines were teetering on the brink of bankruptcy. The board of Lufthansa has recommended



In [ ]:
from langchain_core.prompts import ChatPromptTemplate 

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [19]:
prompt = ChatPromptTemplate.from_template(template)

In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [ ]:
from langchain_core.output_parsers import StrOutputParser 

In [23]:
output_parser = StrOutputParser()

In [26]:
! uv pip install langchain-google-genai

Using Python 3.12.3 environment at: C:\mlops\.venv
Resolved 39 packages in 1.69s
Uninstalled 1 package in 413ms
Installed 1 package in 623ms
 - google-ai-generativelanguage==0.6.15
 + google-ai-generativelanguage==0.9.0


In [ ]:
import os
from dotenv import load_dotenv 
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv() 

gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
    print("FATAL ERROR: GEMINI_API_KEY is not loaded. Check your .env file path and content.")
llm_model = "gemini-2.5-flash"
llm = ChatGoogleGenerativeAI(
    model=llm_model,
    google_api_key=gemini_api_key 
) 

print("LLM initialized successfully!")

LLM initialized successfully!


In [ ]:
from langchain_core.runnables import RunnablePassthrough  

rag_chain = (
    {"context": retreiver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [37]:
rag_chain.invoke("What three specific factors caused Lufthansa's huge losses in 2003?")

'Lufthansa experienced huge losses in 2003 due to three specific factors. The airline was significantly impacted by tough competition in the market. Additionally, there was a substantial dip in demand for air travel following the Iraq war and the outbreak of the killer SARS virus. Finally, Lufthansa was also affected by troubles at its US catering business.'